In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Dimensionality reduction

In [2]:
df = pd.read_csv("../data/mnist-train.csv")

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
len(df)

42000

In [5]:
df=df.sample(1000)

In [6]:
X = df[list(set(df.columns)-set(["label"]))].values
y = df["label"]

In [7]:
# Dimensionality Reduction (TSNE)
from sklearn.manifold import TSNE
red = TSNE(verbose=True)

In [8]:
X_red = red.fit_transform(X)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.028s...
[t-SNE] Computed neighbors for 1000 samples in 1.350s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 575.482515
[t-SNE] KL divergence after 250 iterations with early exaggeration: 70.697792
[t-SNE] Error after 1000 iterations: 1.090455


In [9]:
from sklearn.ensemble import RandomForestClassifier
rand = RandomForestClassifier(max_depth=4)

In [10]:
from sklearn.model_selection import cross_val_score
cross_val_score(rand,X_red,y,cv=10).mean()

0.7228941024365538

In [11]:
cross_val_score(rand,X,y,cv=10).mean()

0.7298989669650054

## Scaling Variables

In [12]:
# http://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py

In [163]:
df = pd.read_csv("../data/pima-indians-diabetes.data",header=None)
X = df[np.arange(8)].values
y = df[8]

In [164]:
from sklearn.preprocessing import RobustScaler

In [165]:
scaler = RobustScaler()
X_scale = scaler.fit_transform(X)

In [166]:
cross_val_score(rand,X_scale,y,cv=10).mean()

0.7551777170198224

In [167]:
cross_val_score(rand,X,y,cv=10).mean()

0.7526315789473684

## Categorical Variables

In [18]:
df = pd.read_csv("../data/mushrooms.csv")
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [19]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

In [20]:
X = df[list(set(df.columns)-set(["class"]))].apply(LabelEncoder().fit_transform).values
y= df["class"]

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
enc = OneHotEncoder()

In [23]:
X_enc=enc.fit_transform(X)

In [24]:
cross_val_score(rand,X_enc,y,cv=10).mean()

0.9430364772205058

In [25]:
## The problem with this approach is that there are too many columns:

In [26]:
len(X[0])

22

In [27]:
len(X_enc.toarray()[0])

117

In [28]:
## Another problem is that there are correlated values in those columns !!!

In [29]:
## Labels

## Another approach is to use weights 

In [155]:
df2 = df.copy()
df2["class"]=LabelEncoder().fit_transform(df2["class"])

In [156]:
for column in list(set(df.columns)-set(["class"])):
    weight = df2.groupby(column).sum()["class"].reset_index()
    weight.columns = [column,"weight"]
    df2[column] = df2[column].reset_index().merge(weight).sort_values("index")["weight"].values
df2.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,1708,1412,1020,624,256,3898,3804,2224,64,...,1536,1712,1680,3916,3908,3808,816,224,368,272
1,0,1708,1412,672,624,0,3898,3804,1692,64,...,1536,1712,1680,3916,3908,3808,816,224,0,740
2,0,48,1412,320,624,0,3898,3804,1692,112,...,1536,1712,1680,3916,3908,3808,816,224,0,36
3,1,1708,1740,320,624,256,3898,3804,2224,112,...,1536,1712,1680,3916,3908,3808,816,224,368,272
4,0,1708,1412,808,3292,120,3898,112,1692,64,...,1536,1712,1680,3916,3908,3808,1768,224,0,740


In [157]:
X_enc=df2[list(set(df.columns)-set(["class"]))].values

In [158]:
cross_val_score(rand,X_enc,y,cv=10).mean()

0.9587921612878274

## Combination with Sklearn Pipeline

In [168]:
df = pd.read_csv("../data/pima-indians-diabetes.data",header=None)
X = df[np.arange(8)].values
y = df[8]

In [169]:
from sklearn.pipeline import Pipeline

In [174]:
from sklearn.linear_model import LogisticRegression

In [175]:
pipe = Pipeline(steps=[('scaler', scaler), ('logistic',LogisticRegression())])

In [178]:
cross_val_score(pipe,X,y,cv=10).mean()

0.7721804511278195

In [179]:
cross_val_score(LogisticRegression(),X,y,cv=10).mean()

0.7669685577580314

## Pipeline do not work with encoder (One-hot and Label) !!!!